In [1]:
#import tuxml
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

#definitions
# systems = ['7z','BerkeleyDBC','Dune','Hipacc','Irzip','LLVM','Polly','x264']
# systems = ['LLVM','x264','BerkeleyDBC','Irzip','Polly','7z','Hipacc','Dune']
systems = ['Irzip']

# samples_config = ['random']
samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']
size_methods = ["Performance"]


In [2]:
#learning a model by using linear regression
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet

def run_regressorML(reg, test_size, size_target, x_train, x_test, y_train, y_test):
    assert(size_target in size_methods)
    reg.fit(x_train, y_train)       
    y_pred = reg.predict(x_test)
    #y_pred = reg.intercept_ + np.sum(reg.coef_ * x_test.values, axis=1)
    dfErrors = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, "error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    return dfErrors["% error"].describe()

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
import pandas as pd

def ft_importances(coef_, col):
    importanceSeries = pd.Series(coef_, index=col.values)
    return importanceSeries[importanceSeries != 0].abs().sort_values(ascending = False)

def ft_importances_RF(X, y):
    model = RandomForestRegressor(random_state=42)
    model.fit(X, y)

    feature_importances = model.feature_importances_
    features = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})
    features.sort_values('importance', ascending=False, inplace=True)
    
    return features

def ft_importances_DT(X, y):
    model = tree.DecisionTreeRegressor(random_state=42)
    model.fit(X, y)

    feature_importances = model.feature_importances_
    features = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})
    features.sort_values('importance', ascending=False, inplace=True)
    
    return features

def ft_importances_GB(X, y):
    model = GradientBoostingRegressor(random_state=42)
    model.fit(X, y)

    feature_importances = model.feature_importances_
    features = pd.DataFrame({'feature': X.columns, 'importance': feature_importances})
    features.sort_values('importance', ascending=False, inplace=True)
    
    return features


In [4]:
df_result  = pd.DataFrame(columns = ['algorithm_name', 'ft_dummyfication', 'origin_ft_selection', 'ft_selection', 'hyperparameters', 'size_target', 'test_size', 'coef_order', 'accuracy'])
df_result

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy


In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn import linear_model
from sklearn import metrics
from sklearn import ensemble
from IPython.core.debugger import set_trace

%run feature_importance-all_systems.ipynb


def run_Algorithms(df, ft_importance_enable, system=None):
    
    samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']
    size_methods = ["Performance"]
    
    for config_name in samples_config:
        
        df_result = pd.DataFrame(columns = ['algorithm_name', 'ft_dummyfication', 'origin_ft_selection', 'ft_selection', 'hyperparameters', 'size_target', 'test_size', 'coef_order', 'accuracy', 'num_features'])
        
        print(config_name)
        path_name = config_name+"_t3"
        if(system==None):
            path = r"sampledConfigurations_"+path_name+".csv"
        else:
            path = system+"/sampledConfigurations_"+path_name+".csv"
            
        if not ft_importance_enable:
            samplePerc_lst = [1.0]
        else:
            samplePerc_lst = [0.3,0.4,0.5,0.6]
            
        for samplePerc in samplePerc_lst:
            print(samplePerc)
            df = pd.read_csv(path, sep=';')
            
            if ft_importance_enable:
                #Run fetures engines: #yes and encoding
                df = run_feature_encoding(run_features_engine_yes(df))

                path_name = config_name+"_t3_features_engine"
            else:
                df = run_features_engine_yes(df)

            
            #display(df)
            #number of features after features engine execution
            num_features = round(samplePerc*len(df.columns))

            start_time_sample = pd.Timestamp.now()
            
            
            #Run feature selection
            if ft_importance_enable:
                run_featureSelection(df, system, None, path_name,'GradientBoostingTree')
                df_importance = pd.read_csv(system+f"/feature_importance_{path_name}_GB.csv")

                print("Sample features number: "+ str(num_features))
                df_importance = df[df_importance[:num_features]["Unnamed: 0"].values]
                df_importance.head()


            target = "Performance"
            paramsLasso = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                           'max_iter': [1, 5, 10, 100, 1000],}
            paramsRidge = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
            paramsENet = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                          'l1_ratio': [0.0, 0.5, 1.0, 0.1],
                          'max_iter': [1, 5, 10, 100, 1000],}
            paramsRF =  {"max_depth":18,"n_estimators":50,"verbose":2}

            #Modifiquei os dicionários para quando a feature importance está ativada. Acho que não fazia muito sentido 
            #manter o uso do random forest quando feature importance for False, da mesma forma que não é viável manter
            #o None quando está ativado
            
            if ft_importance_enable:
                ft_selection =  {'DecisionTree': df_importance.columns[:num_features]}
            else:
                ft_selection =  {'None': df.drop(columns=size_methods).columns}

            for origin_ft_selection in ft_selection:
                print(origin_ft_selection)
                if ft_importance_enable:
                    print('IN FT_LOOP DF IMPORTANCE\n')
                    display(df_importance)

                for size in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
                    config_start_time = pd.Timestamp.now()

                    # size = number of configurations
                    x_train, x_test, y_train, y_test = train_test_split(df[ft_selection[origin_ft_selection]], df[target], train_size=size, random_state=42)
                    x_train, y_train = np.array(x_train.values), np.array(y_train.values)


                    if len(x_train) > 3:

                        model = {'LinearRegression': LinearRegression(),
                                 'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=3).fit(x_train, y_train).best_estimator_,
                                 'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=3).fit(x_train, y_train).best_estimator_,
                                 'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=3).fit(x_train, y_train).best_estimator_,
                                 'PolynomialRegression': LinearRegression(),
                                 'RandomForest': ensemble.RandomForestRegressor(),
                                 'GradientBoostingTree': ensemble.GradientBoostingRegressor(),
                                 'DecisionTree': tree.DecisionTreeRegressor()}
                    else:
                        if len(x_train) >= 2:
                            model = {'LinearRegression': LinearRegression(),
                                     'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=2).fit(x_train, y_train).best_estimator_,
                                     'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=2).fit(x_train, y_train).best_estimator_,
                                     'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=2).fit(x_train, y_train).best_estimator_,
                                     'PolynomialRegression': LinearRegression(),
                                     'RandomForest': ensemble.RandomForestRegressor(),
                                     'GradientBoostingTree': ensemble.GradientBoostingRegressor(),
                                     'DecisionTree': tree.DecisionTreeRegressor()}

                    for key in model:
                        print(key)
                        if (key == 'PolynomialRegression') and (origin_ft_selection == 'None'):
                            print("It doesn't scale")
                        else:
                            if (key == 'PolynomialRegression'):
                                #for 2 options we have: 𝑓(𝑥₁, 𝑥₂) = 𝑏₀ + 𝑏₁𝑥₁ + 𝑏₂𝑥₂ + 𝑏₃𝑥₁² + 𝑏₄𝑥₁𝑥₂ + 𝑏₅𝑥₂²
                                x_train = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_train)
                                x_test = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_test)

                            res_model = run_regressorML(model[key], test_size=size, size_target=target, x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

                            if (key == 'PolynomialRegression'):
                                model[key].coef_ = model[key].coef_[:num_features]
                                
                            if (key == 'DecisionTree' or key == 'RandomForest' or key == 'GradientBoostingTree'):
                                coef_order = ft_importances(res_model[:num_features], col=ft_selection[origin_ft_selection])
                            else:
                                coef_order = ft_importances(model[key].coef_, col=ft_selection[origin_ft_selection])

#                             df_result.loc[len(df_result)] = [key, ft_dummyfication, origin_ft_selection, ft_selection[origin_ft_selection], model[key], target, size, coef_order, res_model]
                            df_result.loc[len(df_result)] = [key, False, origin_ft_selection, ft_selection[origin_ft_selection], model[key], target, size, coef_order, res_model, str(num_features)]
                            display(df_result)
                            
                        config_end_time = pd.Timestamp.now()
                        exec_time = [config_name,samplePerc,num_features,ft_importance_enable,origin_ft_selection, key, size,False,config_start_time,config_end_time]
    #             exec_time = [config_name,samplePerc,num_features,ft_importance_enable,origin_ft_selection, key, size,ft_dummyfication,config_start_time,config_end_time]
                        df_time.loc[df_time.size] = exec_time

            sample_end_time = pd.Timestamp.now()
            exec_time = [config_name,samplePerc,num_features,ft_importance_enable,origin_ft_selection, key, None,None,start_time_sample,sample_end_time]
            df_time.loc[df_time.size] = exec_time
#             display(df_result)

            if ft_importance_enable:
                df_result.to_csv("results/"+system+"/"+system+"-results_with_feature_importance_"+path_name+"_"+str(num_features)+"_v3_GB.csv", header=True)
            else:
                df_result.to_csv("results/"+system+"/"+system+"-results_"+path_name+"_full_v3_GB.csv", header=True)

        display(df_result)
                                                                                                                                  
    if ft_importance_enable:
        time_file_name = "results/"+system+"/"+system+"-simulation_time_information-with_feature_importance_v3_GB.csv"
    else:
        time_file_name = "results/"+system+"/"+system+"-simulation_time_information_v3_GB.csv"
    
    df_time.to_csv(time_file_name)
        
    return df_result

In [6]:
df_time = pd.DataFrame(columns=['SampleAlgorithm','%FeatureSelection','#Features','FeatureImportance','OriginalFS','Algorithm','%Configurations','Dummy','StartTime', 'EndTime'])

for system in systems:
    print(system)
    
    print("\n\n-----------------------------------------------------------------------------")
    print("\n--------------- System "+ system+ " training WITHOUT features importance ")
    print("\n--------------- Start time:: " + str(pd.Timestamp.now()))
    print("\n-----------------------------------------------------------------------------")
    df_result_nfi = run_Algorithms(None, False, system)
    print("\n\n-----------------------------------------------------------------------------")
    print("\n--------------- System "+ system+ " training ")
    print("\n--------------- End time:: " + str(pd.Timestamp.now()))
    print("\n-----------------------------------------------------------------------------")
    
    print("\n\n-----------------------------------------------------------------------------")
    print("\n--------------- System "+ system+ " training WITH features importance ")
    print("\n--------------- Start time:: " + str(pd.Timestamp.now()))
    print("\n-----------------------------------------------------------------------------")
    df_result_fi = run_Algorithms(None, True, system)
    print("\n\n-----------------------------------------------------------------------------")
    print("\n--------------- System "+ system+ " training ")
    print("\n--------------- End time:: " + str(pd.Timestamp.now()))
    print("\n-----------------------------------------------------------------------------")

    

Irzip


-----------------------------------------------------------------------------

--------------- System Irzip training WITHOUT features importance 

--------------- Start time:: 2024-02-09 22:15:57.009422

-----------------------------------------------------------------------------
distBased
1.0
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy,num_features
0,LinearRegression,False,None,"Index(['root', 'encryption', 'compression', 'c...",LinearRegression(),Performance,0.1,compressionLrzip 2.842927e+05 ...,count 225.000000 mean 112.635578 std ...,22
1,Lasso,False,None,"Index(['root', 'encryption', 'compression', 'c...","Lasso(alpha=25, max_iter=1)",Performance,0.1,compressionLzo 244539.746739...,count 225.000000 mean 85.033802 std ...,22
2,Ridge,False,None,"Index(['root', 'encryption', 'compression', 'c...",Ridge(alpha=2),Performance,0.1,compressionLrzip 181675.929638...,count 225.000000 mean 76.208941 std ...,22
3,ElasticNet,False,None,"Index(['root', 'encryption', 'compression', 'c...","ElasticNet(alpha=0.2, max_iter=1)",Performance,0.1,compressionLrzip 137861.873995...,count 225.000000 mean 72.555850 std ...,22
4,RandomForest,False,None,"Index(['root', 'encryption', 'compression', 'c...","(DecisionTreeRegressor(max_features=1.0, rando...",Performance,0.1,root NaN encryption...,count 225.000000 mean 30.599375 std ...,22
...,...,...,...,...,...,...,...,...,...,...
58,Ridge,False,None,"Index(['root', 'encryption', 'compression', 'c...",Ridge(alpha=2),Performance,0.9,compressionZpaq 1.819683e+06 ...,count 25.000000 mean 249.435121 std ...,22
59,ElasticNet,False,None,"Index(['root', 'encryption', 'compression', 'c...","ElasticNet(alpha=0.02, l1_ratio=0.1, max_iter=5)",Performance,0.9,compressionZpaq 1.689284e+06 ...,count 25.000000 mean 226.158701 std ...,22
60,RandomForest,False,None,"Index(['root', 'encryption', 'compression', 'c...","(DecisionTreeRegressor(max_features=1.0, rando...",Performance,0.9,root NaN encryption...,count 25.000000 mean 3.439838 std ...,22
61,GradientBoostingTree,False,None,"Index(['root', 'encryption', 'compression', 'c...",([DecisionTreeRegressor(criterion='friedman_ms...,Performance,0.9,root NaN encryption...,count 25.000000 mean 23.245059 std ...,22


divDistBased
1.0
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingT

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy,num_features
0,LinearRegression,False,None,"Index(['root', 'encryption', 'compression', 'c...",LinearRegression(),Performance,0.1,compressionZpaq 1.996469e+06 ...,count 225.000000 mean 879.475208 std ...,22
1,Lasso,False,None,"Index(['root', 'encryption', 'compression', 'c...","Lasso(alpha=25, max_iter=5)",Performance,0.1,compressionZpaq 1.863784e+06 ...,count 225.000000 mean 850.035511 std ...,22
2,Ridge,False,None,"Index(['root', 'encryption', 'compression', 'c...",Ridge(),Performance,0.1,compressionZpaq 1.628442e+06 ...,count 225.000000 mean 588.498146 std ...,22
3,ElasticNet,False,None,"Index(['root', 'encryption', 'compression', 'c...","ElasticNet(alpha=0.3, max_iter=1)",Performance,0.1,compressionZpaq 1.428918e+06 ...,count 225.000000 mean 371.201087 std ...,22
4,RandomForest,False,None,"Index(['root', 'encryption', 'compression', 'c...","(DecisionTreeRegressor(max_features=1.0, rando...",Performance,0.1,root NaN encryption...,count 225.000000 mean 174.851473 std ...,22
...,...,...,...,...,...,...,...,...,...,...
58,Ridge,False,None,"Index(['root', 'encryption', 'compression', 'c...",Ridge(alpha=2),Performance,0.9,compressionZpaq 1.746594e+06 ...,count 25.000000 mean 258.839644 std ...,22
59,ElasticNet,False,None,"Index(['root', 'encryption', 'compression', 'c...","ElasticNet(alpha=0.02, l1_ratio=0.1, max_iter=5)",Performance,0.9,compressionZpaq 1.601062e+06 ...,count 25.000000 mean 238.285845 std ...,22
60,RandomForest,False,None,"Index(['root', 'encryption', 'compression', 'c...","(DecisionTreeRegressor(max_features=1.0, rando...",Performance,0.9,root NaN encryption...,count 25.000000 mean 5.222441 std ...,22
61,GradientBoostingTree,False,None,"Index(['root', 'encryption', 'compression', 'c...",([DecisionTreeRegressor(criterion='friedman_ms...,Performance,0.9,root NaN encryption...,count 25.000000 mean 10.624310 std ...,22


henard
1.0
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
It doesn't scale
RandomForest
GradientBoostingTree
De

...

Sample features number: 10
IN FT_LOOP DF IMPORTANCE

LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientB

['root', 'level']

Sample features number: 12
IN FT_LOOP DF IMPORTANCE

LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientBoostingTree
DecisionTree
LinearRegression
Lasso
Ridge
ElasticNet
PolynomialRegression
RandomForest
GradientB

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy,num_features
0,LinearRegression,False,GradientBoostingTree,"Index(['compressionZpaq', 'level8', 'level9', ...",LinearRegression(),Performance,0.1,compressionZpaq 2.095620e+06 level8 ...,count 161.000000 mean 145.842735 std ...,6
1,Lasso,False,GradientBoostingTree,"Index(['compressionZpaq', 'level8', 'level9', ...","Lasso(alpha=25, max_iter=10)",Performance,0.1,compressionZpaq 2.095668e+06 level8 ...,count 161.000000 mean 145.736298 std ...,6
2,Ridge,False,GradientBoostingTree,"Index(['compressionZpaq', 'level8', 'level9', ...",Ridge(alpha=25),Performance,0.1,compressionZpaq 74462.390735 level9 ...,count 161.000000 mean 236.677391 std ...,6
3,ElasticNet,False,GradientBoostingTree,"Index(['compressionZpaq', 'level8', 'level9', ...","ElasticNet(alpha=25, l1_ratio=0.0, max_iter=1)",Performance,0.1,compressionZpaq 4569.454495 level9 ...,count 161.000000 mean 235.239518 std ...,6
4,PolynomialRegression,False,GradientBoostingTree,"Index(['compressionZpaq', 'level8', 'level9', ...",LinearRegression(),Performance,0.1,compressionZpaq 1.047810e+06 level8 ...,count 161.000000 mean 145.842735 std ...,6
...,...,...,...,...,...,...,...,...,...,...
283,ElasticNet,False,GradientBoostingTree,"Index(['compressionZpaq', 'level8', 'level9', ...","ElasticNet(alpha=0.1, l1_ratio=0.1, max_iter=1)",Performance,0.9,compressionZpaq 1.399714e+06 ...,count 18.000000 mean 121.487787 std ...,12
284,PolynomialRegression,False,GradientBoostingTree,"Index(['compressionZpaq', 'level8', 'level9', ...",LinearRegression(),Performance,0.9,compressionZpaq 8.933683e+17 ...,count 18.000000 mean 5.859608 std ...,12
285,RandomForest,False,GradientBoostingTree,"Index(['compressionZpaq', 'level8', 'level9', ...","(DecisionTreeRegressor(max_features=1.0, rando...",Performance,0.9,compressionZpaq NaN level8 ...,count 18.000000 mean 15.427236 std ...,12
286,GradientBoostingTree,False,GradientBoostingTree,"Index(['compressionZpaq', 'level8', 'level9', ...",([DecisionTreeRegressor(criterion='friedman_ms...,Performance,0.9,compressionZpaq NaN level8 ...,count 18.000000 mean 10.418345 std ...,12




-----------------------------------------------------------------------------

--------------- System Irzip training 

--------------- End time:: 2024-02-09 22:23:48.229328

-----------------------------------------------------------------------------


In [7]:
df_result['accuracy'][0]

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

rs = []
for i in range(len(df_result_fi['accuracy'])):
    rs.append(df_result_fi['accuracy'][i]['mean'])

# rs
# display (df_result['algorithm_name'], df_result['accuracy'], rs)

plt.scatter(df_result_fi['algorithm_name'], rs, color ="blue", marker = "s")
plt.plot(df_result_fi['algorithm_name'], rs, color ="blue")


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# Boxplot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


data = [model['LinearRegression'].coef_, model['Lasso'].coef_, model['Ridge'].coef_, model['ElasticNet'].coef_, model['PolynomialRegression'].coef_]
fig1, ax1 = plt.subplots()
ax1.set_title('Coeficient of feature importance for all algorithms')
ax1.boxplot(data)
plt.show()